I will build a regression model to predict the listing price of houses to be listed on the Airbnb websits. Model performance will be guided by RMSE, and the model should at least imporve upon baseline by 10%. 
Baseline is defined as the average daily costs per night for a housing listing in Singapore.

In [1]:
import pandas as pd

In [2]:
airbnb_data = pd.read_csv('../data/listings.csv')

In [3]:
airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8047 entries, 0 to 8046
Columns: 106 entries, id to reviews_per_month
dtypes: float64(20), int64(23), object(63)
memory usage: 6.5+ MB


In [4]:
airbnb_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

In [5]:
airbnb_data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,49091,https://www.airbnb.com/rooms/49091,20200227171146,2020-02-28,COZICOMFORT LONG TERM STAY ROOM 2,NaN,This is Room No. 2.(available after Jan 15 201...,This is Room No. 2.(available after Jan 15 201...,none,NaN,...,f,f,flexible,t,t,2,0,2,0,0.01
1,50646,https://www.airbnb.com/rooms/50646,20200227171146,2020-02-27,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,"Good Condition room, bright & windy. Comes wit...",Fully furnished bedroom with a nice view on th...,none,The serenity & quiet surrounding makes it an i...,...,f,f,moderate,f,t,1,0,1,0,0.25
2,56334,https://www.airbnb.com/rooms/56334,20200227171146,2020-02-28,COZICOMFORT,NaN,This is Room no. 1. USD700 per month On the fa...,This is Room no. 1. USD700 per month On the fa...,none,NaN,...,f,f,flexible,t,t,2,0,2,0,0.19
3,71609,https://www.airbnb.com/rooms/71609,20200227171146,2020-02-28,Ensuite Room (Room 1 & 2) near EXPO,For 3 (Website hidden by Airbnb) room 1&2 and ...,Landed Homestay Room for Rental. Between Simei...,For 3 (Website hidden by Airbnb) room 1&2 and ...,none,NaN,...,t,f,flexible,f,t,8,0,8,0,0.20
4,71896,https://www.airbnb.com/rooms/71896,20200227171146,2020-02-28,B&B Room 1 near Airport & EXPO,NaN,Vocational Stay Deluxe Bedroom in Singapore.(N...,Vocational Stay Deluxe Bedroom in Singapore.(N...,none,NaN,...,t,f,moderate,f,t,8,0,8,0,0.23


In [6]:
airbnb_data.drop_duplicates(subset = 'name', inplace = True)

In [7]:
airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7429 entries, 0 to 8046
Columns: 106 entries, id to reviews_per_month
dtypes: float64(20), int64(23), object(63)
memory usage: 6.1+ MB


In [10]:
null_values=airbnb_data.isnull().sum().sort_values(ascending=False)

In [16]:
null_values_df = pd.DataFrame(data=null_values, columns=['no.'])

In [20]:
null_values_df.head(50)

,no.
jurisdiction_names,7429
xl_picture_url,7429
medium_url,7429
thumbnail_url,7429
license,7423
square_feet,7387
weekly_price,6948
monthly_price,6867
state,6843
house_rules,3767
